<a href="https://colab.research.google.com/github/victordot/Inteligencia-Artificial-2025-2/blob/main/04-modelo%20con%20preprocesado%20Embedding%20Layers%20y%20Red%20Profunda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"victordocampot","key":"0f8ade079b2df30ff64919a0ea693ea6"}'}

In [ ]:
import os
import shutil

# Crear carpeta oculta .kaggle
os.makedirs('/root/.kaggle', exist_ok=True)

source_path = "kaggle.json"
destination_path = "/root/.kaggle/kaggle.json"

if os.path.exists(destination_path):
    os.remove(destination_path)

shutil.move(source_path, destination_path)


'/root/.kaggle/kaggle.json'

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia
!ls -lh

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.36GB/s]
total 30M
drwxr-xr-x 1 root root 4.0K Nov 20 14:30 sample_data
-rw-r--r-- 1 root root  30M Sep 16 01:46 udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip


In [ ]:
!pip install category_encoders xgboost scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 5.8 MB/s eta 0:00:00


In [ ]:
import zipfile

with zipfile.ZipFile("udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip", "r") as zip_ref:
    zip_ref.extractall("dataset_saber_pro")

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from category_encoders import TargetEncoder
import xgboost as xgb

# Cargar datos

df = pd.read_csv('/content/dataset_saber_pro/train.csv')

# Verificamos la carga
print(f"Dimensiones del dataset: {df.shape}")
df.head()

Dimensiones del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Cargar datos
df = pd.read_csv('/content/dataset_saber_pro/train.csv')
df_test = pd.read_csv('/content/dataset_saber_pro/test.csv')

#LIMPIEZA BÁSICA
def basic_cleaning(dframe):
    df_c = dframe.copy()
    # Eliminar ID y duplicados
    df_c = df_c.drop(columns=['ID', 'F_TIENEINTERNET.1'], errors='ignore')
    # Mapear Target si existe
    if 'RENDIMIENTO_GLOBAL' in df_c.columns:
        target_map = {'bajo': 0, 'medio-bajo': 1, 'medio-alto': 2, 'alto': 3}
        df_c['RENDIMIENTO_GLOBAL'] = df_c['RENDIMIENTO_GLOBAL'].map(target_map)
    return df_c

df_clean = basic_cleaning(df)
df_test_clean = basic_cleaning(df_test)

# Separar Target
y = df_clean.pop('RENDIMIENTO_GLOBAL')
X = df_clean

#PREPROCESAMIENTO

# 1. Definir Grupos de Columnas
# Categóricas para Embeddings (Alta y Baja cardinalidad)
cat_cols = [col for col in X.columns if X[col].dtype == 'object']

# Numéricas
num_cols = [col for col in X.columns if X[col].dtype != 'object']

# 2. Manejo de Numéricas (Imputar + Escalar)
num_imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[num_cols] = scaler.fit_transform(X[num_cols])
# Aplicar lo mismo al test
df_test_clean[num_cols] = num_imputer.transform(df_test_clean[num_cols])
df_test_clean[num_cols] = scaler.transform(df_test_clean[num_cols])

# 3. Manejo de Categóricas
cat_dims = {}

for col in cat_cols:

    X[col] = X[col].fillna("MISSING").astype(str)
    df_test_clean[col] = df_test_clean[col].fillna("MISSING").astype(str)

    # Ajustar LabelEncoder en ambos conjuntos
    le = LabelEncoder()
    # Unimos train y test solo para ajustar el encoder y conocer todas las categorias posibles
    all_values = pd.concat([X[col], df_test_clean[col]], axis=0)
    le.fit(all_values)

    X[col] = le.transform(X[col])
    df_test_clean[col] = le.transform(df_test_clean[col])

    # Guardamos el número de categorías únicas (+1 por seguridad)
    cat_dims[col] = len(le.classes_) + 1

# Dividir Train/Val
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Datos listos. Numéricas escaladas y Categóricas convertidas a Enteros.")

Datos listos. Numéricas escaladas y Categóricas convertidas a Enteros.


In [ ]:
def build_model(cat_cols, num_cols, cat_dims):
    inputs = []
    embeddings = []

    # 1. Ramas para variables Categóricas (Embeddings)
    for col in cat_cols:
        # Input layer para cada columna categórica
        input_cat = keras.Input(shape=(1,), name=f"in_{col}")
        inputs.append(input_cat)

        # Dimensión del vector de embedding (Regla empírica: min(50, categorias/2))
        no_of_unique_cat = cat_dims[col]
        embedding_size = min(np.ceil((no_of_unique_cat)/2), 50)
        embedding_size = int(embedding_size)

        # Capa de Embedding
        emb = layers.Embedding(input_dim=no_of_unique_cat, output_dim=embedding_size)(input_cat)
        # Aplanar el vector (Reshape)
        emb = layers.Flatten()(emb)
        embeddings.append(emb)

    # 2. Rama para variables Numéricas
    if len(num_cols) > 0:
        input_num = keras.Input(shape=(len(num_cols),), name="in_numerical")
        inputs.append(input_num)
        embeddings.append(input_num)

    # 3. Concatenar todo
    x = layers.Concatenate()(embeddings)

    # 4. Capas Densas (Deep Learning)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)   # Ayuda a estabilizar el entrenamiento
    x = layers.Dropout(0.3)(x)           # Apaga 30% neuronas para evitar overfitting

    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Dense(64, activation='relu')(x)

    # 5. Capa de Salida
    output = layers.Dense(4, activation='softmax', name="output")(x)

    model = keras.Model(inputs=inputs, outputs=output)
    return model

# Construir modelo
model_dl = build_model(cat_cols, num_cols, cat_dims)

# Compilar
model_dl.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import AdamW

# Función auxiliar para preparar el formato de entrada de Keras
def prepare_inputs(df_in, cat_cols, num_cols):
    inputs_dict = {}
    # Entradas categóricas una por una
    for col in cat_cols:
        inputs_dict[f"in_{col}"] = df_in[col].values

    # Entrada numérica en bloque
    if len(num_cols) > 0:
        inputs_dict["in_numerical"] = df_in[num_cols].values

    return inputs_dict

train_inputs = prepare_inputs(X_train, cat_cols, num_cols)
val_inputs = prepare_inputs(X_val, cat_cols, num_cols)
def build_tuned_model(cat_cols, num_cols, cat_dims):
    inputs = []
    embeddings = []

    #Ramas de Embeddings
    for col in cat_cols:
        input_cat = keras.Input(shape=(1,), name=f"in_{col}")
        inputs.append(input_cat)

        no_of_unique_cat = cat_dims[col]

        embedding_size = min(np.ceil((no_of_unique_cat)/1.6), 60)
        embedding_size = int(embedding_size)

        emb = layers.Embedding(input_dim=no_of_unique_cat, output_dim=embedding_size)(input_cat)
        emb = layers.Flatten()(emb)
        embeddings.append(emb)

    #Rama Numérica
    if len(num_cols) > 0:
        input_num = keras.Input(shape=(len(num_cols),), name="in_numerical")
        inputs.append(input_num)
        embeddings.append(input_num)

    #Concatenación
    x = layers.Concatenate()(embeddings)

    # Capa 1: Más ancha, activación Swish, Regularización L2 leve
    x = layers.Dense(512, activation='swish', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    # Capa 2
    x = layers.Dense(256, activation='swish', kernel_regularizer=regularizers.l2(0.0001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Capa 3
    x = layers.Dense(128, activation='swish')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    # Salida
    output = layers.Dense(4, activation='softmax', name="output")(x)

    model = keras.Model(inputs=inputs, outputs=output)
    return model

# Construir el modelo afinado
model_tuned = build_tuned_model(cat_cols, num_cols, cat_dims)

# Compilar con AdamW
model_tuned.compile(
    optimizer=AdamW(learning_rate=0.001, weight_decay=0.004),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

#DEFINICIÓN DE CALLBACKS

# ReduceLROnPlateau: Si la val_loss no mejora en 3 épocas, divide el learning rate por 5.
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,           # Reducir LR al 20% del actual
    patience=3,           # Esperar 3 épocas sin mejora
    min_lr=1e-6,          # Límite inferior
    verbose=1             # Mostrar mensaje cuando ocurra
)


early_stopping = EarlyStopping(
    patience=10,
    restore_best_weights=True,
    monitor='val_loss'
)

print("Iniciando entrenamiento con Ajuste Fino (Scheduler + AdamW)...")

history_tuned = model_tuned.fit(
    train_inputs,
    y_train,
    validation_data=(val_inputs, y_val),
    epochs=50,           # Aumentamos épocas potenciales
    batch_size=512,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

Iniciando entrenamiento con Ajuste Fino (Scheduler + AdamW)...
Epoch 1/50
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 54s 44ms/step - accuracy: 0.3835 - loss: 1.3982 - val_accuracy: 0.4261 - val_loss: 1.2410 - learning_rate: 0.0010
Epoch 2/50
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 47s 43ms/step - accuracy: 0.4265 - loss: 1.2360 - val_accuracy: 0.4318 - val_loss: 1.2188 - learning_rate: 0.0010
Epoch 3/50
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 47s 44ms/step - accuracy: 0.4320 - loss: 1.2174 - val_accuracy: 0.4351 - val_loss: 1.2094 - learning_rate: 0.0010
Epoch 4/50
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 46s 42ms/step - accuracy: 0.4346 - loss: 1.2119 - val_accuracy: 0.4364 - val_loss: 1.2078 - learning_rate: 0.0010
Epoch 5/50
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 50s 46ms/step - accuracy: 0.4340 - loss: 1.2087 - val_accuracy: 0.4368 - val_loss: 1.2062 - learning_rate: 0.0010
Epoch 6/50
1083/1083 ━━━━━━━━━━━━━━━━━━━━ 47s 44ms/step - accuracy: 0.4353 - loss: 1.2079 - val_accuracy: 0.4367 - val_loss: 1.2044 - learning_rate: 0.0010
E

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Generar Predicciones en el Conjunto de Prueba ---

# Asumimos que las variables df_test_clean, cat_cols, num_cols y prepare_inputs
# están definidas con los datos de test ya preprocesados (Integer Encoding, Scaling).

# 1.1 Formatear la entrada de prueba para Keras
test_inputs = prepare_inputs(df_test_clean, cat_cols, num_cols)

print("Generando predicciones de probabilidad en el conjunto de prueba...")
# Predecir probabilidades para las 4 clases
probs_tuned = model_tuned.predict(test_inputs)

# 1.2 Obtener la clase final
# np.argmax selecciona el índice con la mayor probabilidad (la predicción final)
preds_idx_tuned = np.argmax(probs_tuned, axis=1)

# --- 2. Convertir a Etiquetas de Texto ---

# Mapeo inverso de los números (0, 1, 2, 3) a las etiquetas de texto originales
reverse_map = {0: 'bajo', 1: 'medio-bajo', 2: 'medio-alto', 3: 'alto'}
preds_labels_tuned = [reverse_map[i] for i in preds_idx_tuned]

# --- 3. Crear y Guardar Archivo de Envío ---

# Cargar los IDs originales del archivo de prueba para el submission
try:
    test_ids = pd.read_csv('/content/dataset_saber_pro/test.csv')['ID']
except FileNotFoundError:
    print("Error: Asegúrate de que 'test.csv' esté accesible para obtener los IDs.")

submission_tuned = pd.DataFrame({
    'ID': test_ids,
    'RENDIMIENTO_GLOBAL': preds_labels_tuned
})

# Guardar el archivo CSV
submission_tuned.to_csv('submission_tuned_dl.csv', index=False)

print("\n🎉 Archivo 'submission_tuned_dl.csv' generado exitosamente.")
print(f"Predicciones generadas para {len(submission_tuned)} estudiantes.")

Generando predicciones de probabilidad en el conjunto de prueba...
9275/9275 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step

🎉 Archivo 'submission_tuned_dl.csv' generado exitosamente.
Predicciones generadas para 296786 estudiantes.
